In [1]:
import json
import pandas as pd
import numpy as np
import glob
import csv
import datetime
from sqlalchemy import create_engine
import psycopg2
from config import db_password
import time

In [2]:
predicted_df = pd.read_csv('https://gentrificationoutcome.s3.amazonaws.com/predicted.csv')
predicted_df.set_index('zip', inplace=True)
predicted_df.head()

,Index Change,Percent_Change_House,Percent_Change_Rent,Percent_Change_Income,Percent_Change_Public_Transp,Percent_Change_Population,Percent_Change_White,Outcome
zip,,,,,,,,
91311,0.28,0.418051,0.592135,0.289208,0.011,0.069428,-0.102,0
90670,0.78,0.492026,0.699865,0.127527,0.012,-0.041136,0.075,0
92065,-0.10,0.400943,0.728412,0.280726,-0.004,0.089219,0.030,0
93710,0.07,0.332705,0.561279,0.272468,0.014,0.129130,-0.023,0
95019,0.94,0.561080,0.627803,0.002944,0.006,0.557668,0.218,0


In [3]:
california_df =pd.read_csv('https://gentrificationmldata.s3.amazonaws.com/california_blank_tagged.csv')
california_df.set_index('zip', inplace=True)
california_df.head()

,median_house_value_2000,median_house_value_2014,percent_change_house,median_income_2000,median_income_2014,percent_change_income,median_rent_2000,median_rent_2014,percent_change_rent,average_education_index_2000,...,percent_public_transp_2000,percent_public_transp_2014,percent_change_public_tranp,population_2000,population_2014,percent_change_population,percent_white_2000,percent_white_2014,percent_change_white,Outcome
zip,,,,,,,,,,,,,,,,,,,,,
90001,138900,241800,0.740821,24207,34050,0.41,568,975,0.717,9.49,...,0.172,0.167,-0.029070,54481,56314,0.034,0.246,0.499,1.028,0
90002,127700,218800,0.713391,22872,30214,0.32,560,968,0.729,10.06,...,0.123,0.137,0.113821,44584,50098,0.124,0.166,0.484,1.916,0
90003,134400,228600,0.700893,22346,30016,0.34,572,1064,0.860,9.91,...,0.173,0.175,0.011561,58187,66913,0.150,0.193,0.224,0.161,0
90004,371100,724900,0.953382,27591,38493,0.40,600,1070,0.783,11.83,...,0.216,0.231,0.069444,67850,63547,-0.063,0.338,0.345,0.021,0
90005,392200,635500,0.620347,21998,31214,0.42,547,939,0.717,11.37,...,0.342,0.351,0.026316,43014,38638,-0.102,0.278,0.196,-0.295,0


In [4]:
pred_copy = predicted_df.copy()
cali_copy = california_df.copy()

In [5]:
pred_copy.Outcome = pred_copy.Outcome.replace({1:2})

In [9]:
pred_copy.Outcome.unique()

array([0, 2])

In [10]:
cali_copy.update(pred_copy)

In [11]:
cali_copy.Outcome.unique()

array([0., 2., 1.])

In [13]:
cali_copy.Outcome.count()

1090

In [22]:
new_df = cali_copy.astype({'Outcome':'int32'})

In [24]:
new_df

,median_house_value_2000,median_house_value_2014,percent_change_house,median_income_2000,median_income_2014,percent_change_income,median_rent_2000,median_rent_2014,percent_change_rent,average_education_index_2000,...,percent_public_transp_2000,percent_public_transp_2014,percent_change_public_tranp,population_2000,population_2014,percent_change_population,percent_white_2000,percent_white_2014,percent_change_white,Outcome
zip,,,,,,,,,,,,,,,,,,,,,
90001,138900,241800,0.740821,24207,34050,0.41,568,975,0.717,9.49,...,0.172,0.167,-0.029070,54481,56314,0.034,0.246,0.499,1.028,0
90002,127700,218800,0.713391,22872,30214,0.32,560,968,0.729,10.06,...,0.123,0.137,0.113821,44584,50098,0.124,0.166,0.484,1.916,0
90003,134400,228600,0.700893,22346,30016,0.34,572,1064,0.860,9.91,...,0.173,0.175,0.011561,58187,66913,0.150,0.193,0.224,0.161,2
90004,371100,724900,0.953382,27591,38493,0.40,600,1070,0.783,11.83,...,0.216,0.231,0.069444,67850,63547,-0.063,0.338,0.345,0.021,0
90005,392200,635500,0.620347,21998,31214,0.42,547,939,0.717,11.37,...,0.342,0.351,0.026316,43014,38638,-0.102,0.278,0.196,-0.295,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96141,350000,509100,0.454571,75863,57212,-0.25,725,1221,0.684,14.51,...,0.025,0.089,2.560000,840,669,-0.204,0.973,0.937,-0.037,0
96143,222800,377100,0.692549,38366,40340,0.05,688,985,0.432,12.60,...,0.097,0.043,-0.556701,4802,3855,-0.197,0.740,0.900,0.216,0
96145,381000,589300,0.546719,54545,68887,0.26,885,1340,0.514,14.54,...,0.002,0.062,30.000000,3997,3215,-0.196,0.956,0.956,0.000,1


In [26]:
new_df.to_csv('final.csv', index='zip', header=True)